# Case Study: 7-Day Call Volume Forecast with Prophet


## A Deep Dive into Facebook Prophet for Contact Centre Workforce Planning

---


## Why Prophet for This Problem?

Prophet was designed by Facebook's Core Data Science team specifically for the class of problem you face every Monday morning: business time series with strong seasonal patterns, known holiday effects, and the need for analyst-interpretable forecasts that non-technical stakeholders can trust.

For daily call volume forecasting at a retail bank, Prophet's architecture maps almost perfectly onto the data-generating process:

- **Piecewise linear or logistic growth trend** → captures the slow drift in call volume as customers migrate to digital channels
- **Fourier-based seasonality** → handles both weekly and annual cycles simultaneously (something SARIMA cannot do natively)
- **Built-in holiday framework** → models the exact effect of Irish bank holidays, including the post-holiday rebound
- **Custom regressors** → incorporates banking-specific drivers like direct debit dates and marketing campaigns
- **Automatic changepoint detection** → adapts to structural shifts (branch closures, new IVR routing, COVID-era changes)
- **Uncertainty intervals** → feeds directly into risk-adjusted staffing decisions

This case study works through the full end-to-end workflow with Prophet as the primary model, showing every line of code you need to go from raw data to a production-ready 7-day forecast.

---


## Stage 1: EXPLORE

### 1.1 Load and Prepare the Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (16, 5)
plt.rcParams['font.size'] = 12

# ──────────────────────────────────────────────────
# Load data
# ──────────────────────────────────────────────────
# Your data should have at minimum: date, daily_call_volume
# Additional columns (if available): channel, queue, call_type
df = pd.read_csv('daily_call_volume.csv', parse_dates=['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f'Date range : {df["date"].min()} to {df["date"].max()}')
print(f'Observations: {len(df)}')
print(f'Missing dates: {pd.date_range(df["date"].min(), df["date"].max()).difference(df["date"]).shape[0]}')
print(f'\nFirst 5 rows:')
print(df.head())
print(f'\nDescriptive statistics:')
print(df['calls'].describe())


### 1.2 Plot the Full History


In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
ax.plot(df['date'], df['calls'], linewidth=0.5, color='#2c3e50')
ax.set_title('Daily Call Volume — Full History', fontsize=14, fontweight='bold')
ax.set_ylabel('Calls')
ax.set_xlabel(None)
plt.tight_layout()
plt.show()


**What you are documenting at this stage:**

Read the plot systematically — trend direction, seasonal patterns (weekly sawtooth, annual wave), variance behaviour (constant or growing), and any visible anomalies (spikes, drops, flat periods). Write down your observations before running any statistical tests. The visual inspection guides every decision that follows.

### 1.3 Weekly Pattern Deep Dive


In [ ]:
df['day_of_week'] = df['date'].dt.day_name()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Box plot
sns.boxplot(data=df, x='day_of_week', y='calls', order=dow_order,
            palette='coolwarm', ax=axes[0])
axes[0].set_title('Volume Distribution by Day of Week', fontsize=13, fontweight='bold')
axes[0].set_xlabel(None)
axes[0].tick_params(axis='x', rotation=45)

# Mean with confidence interval
dow_means = df.groupby('day_of_week')['calls'].agg(['mean', 'std', 'count']).reindex(dow_order)
dow_means['se'] = dow_means['std'] / np.sqrt(dow_means['count'])
axes[1].bar(range(7), dow_means['mean'], yerr=1.96 * dow_means['se'],
            color='#3498db', capsize=4, alpha=0.8)
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[1].set_title('Mean Volume ± 95% CI by Day of Week', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Calls')

plt.tight_layout()
plt.show()

# Print the day-of-week profile as percentages (useful later for validation)
dow_pct = dow_means['mean'] / dow_means['mean'].sum() * 100
print('Day-of-week profile (% of weekly volume):')
for day, pct in dow_pct.items():
    print(f'  {day:12s}: {pct:.1f}%')


### 1.4 Monthly and Annual Patterns


In [ ]:
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Aggregate to weekly to smooth out day-of-week noise before looking at annual pattern
weekly = df.set_index('date')['calls'].resample('W').mean().reset_index()
weekly['month'] = weekly['date'].dt.month

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

sns.boxplot(data=weekly, x='month', y='calls', palette='viridis', ax=axes[0])
axes[0].set_title('Weekly-Averaged Volume by Month', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Avg Daily Calls')

# Year-over-year overlay (seasonal plot)
for year, group in df.groupby('year'):
    group_weekly = group.set_index('date')['calls'].resample('W').mean()
    axes[1].plot(range(len(group_weekly)), group_weekly.values, label=str(year), alpha=0.7)
axes[1].set_title('Year-over-Year Weekly Average', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Week of Year')
axes[1].set_ylabel('Avg Daily Calls')
axes[1].legend(title='Year')

plt.tight_layout()
plt.show()


### 1.5 Decomposition


In [ ]:
# Set up as a properly indexed series for MSTL
ts = df.set_index('date')['calls'].asfreq('D')
ts = ts.interpolate(method='linear')  # Fill any gaps for decomposition

# MSTL: weekly (7) and annual (365) periods
mstl = MSTL(ts, periods=[7, 365], stl_kwargs={'robust': True})
result = mstl.fit()

fig = result.plot()
fig.set_size_inches(16, 12)
fig.suptitle('MSTL Decomposition (Weekly + Annual Seasonality)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

# Quantify strength
var_resid = np.var(result.resid)
print(f'Strength of trend           : {max(0, 1 - var_resid / np.var(result.resid + result.trend - result.trend.mean())):.3f}')
for col in result.seasonal.columns:
    F_s = max(0, 1 - var_resid / np.var(result.resid + result.seasonal[col]))
    print(f'Strength of seasonality ({col}): {F_s:.3f}')


### 1.6 ACF/PACF


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
plot_acf(ts.dropna(), lags=60, ax=axes[0], title='ACF: Daily Call Volume')
plot_pacf(ts.dropna(), lags=60, method='ywm', ax=axes[1], title='PACF: Daily Call Volume')
plt.tight_layout()
plt.show()


**Expected signatures:** Spikes at lags 7, 14, 21, 28 in the ACF (weekly cycle). Strong PACF spikes at lags 1 and 7. Slow ACF decay if there is a trend component.

### 1.7 Stationarity


In [ ]:
adf_result = adfuller(ts.dropna(), autolag='AIC')
kpss_result = kpss(ts.dropna(), regression='ct', nlags='auto')

print(f'ADF  — statistic: {adf_result[0]:.4f}, p-value: {adf_result[1]:.4f}')
print(f'KPSS — statistic: {kpss_result[0]:.4f}, p-value: {kpss_result[1]:.4f}')


**Note for Prophet users:** Prophet does not require stationarity. It models trend and seasonality explicitly as additive components, so you do not need to difference the data. However, understanding whether the series is stationary still informs your choice of `growth` parameter (linear vs logistic vs flat) and whether changepoints are likely.

### 1.8 EDA Summary

| Aspect | Typical Finding for Bank Call Data |
|---|---|
| Primary seasonality | Weekly (period=7), very strong |
| Secondary seasonality | Annual (period≈365), moderate |
| Trend | Slight downward (digital migration) or flat |
| Variance | Roughly constant (additive seasonality likely) |
| Key external drivers | Bank holidays, direct debit dates, campaigns, outages |
| ACF signature | Strong at lags 7, 14, 21; also lag 1 |
| Stationarity | Often stationary or trend-stationary |
| Outliers | Holiday rebounds, system incidents, COVID period |

---


## Stage 2: PREPROCESS

### 2.1 Prepare the Prophet DataFrame

Prophet requires a DataFrame with exactly two columns: `ds` (datestamp) and `y` (target). Additional columns are regressors.


In [ ]:
from prophet import Prophet

# ──────────────────────────────────────────────────
# Core DataFrame
# ──────────────────────────────────────────────────
prophet_df = df[['date', 'calls']].copy()
prophet_df = prophet_df.rename(columns={'date': 'ds', 'calls': 'y'})

# Ensure continuous daily index (Prophet can handle missing values but
# performs better with a complete index)
full_dates = pd.DataFrame({'ds': pd.date_range(prophet_df['ds'].min(), prophet_df['ds'].max(), freq='D')})
prophet_df = full_dates.merge(prophet_df, on='ds', how='left')

# Check for missing values
n_missing = prophet_df['y'].isna().sum()
print(f'Missing values: {n_missing}')
if n_missing > 0:
    print('Filling with linear interpolation (acceptable for small gaps)')
    prophet_df['y'] = prophet_df['y'].interpolate(method='linear')


### 2.2 Handle Outliers

Prophet is sensitive to outliers — a single anomalous day can distort the trend changepoints. The recommended approach is to set outlier values to `NaN` and let Prophet impute them:


In [ ]:
# ──────────────────────────────────────────────────
# Flag and remove data quality outliers
# (NOT holiday effects — those are modelled explicitly)
# ──────────────────────────────────────────────────

# Method: IQR within each day-of-week group
def flag_outliers(group, k=3.0):
    q1, q3 = group['y'].quantile([0.25, 0.75])
    iqr = q3 - q1
    group['is_outlier'] = (group['y'] < q1 - k * iqr) | (group['y'] > q3 + k * iqr)
    return group

prophet_df['dow'] = prophet_df['ds'].dt.dayofweek
prophet_df = prophet_df.groupby('dow', group_keys=False).apply(flag_outliers)

n_outliers = prophet_df['is_outlier'].sum()
print(f'Flagged {n_outliers} outliers ({n_outliers / len(prophet_df) * 100:.1f}%)')

# Review flagged outliers
print(prophet_df[prophet_df['is_outlier']][['ds', 'y']].head(20))

# Set outliers to NaN — Prophet handles NaN by imputing during fitting
# IMPORTANT: Only do this for genuine data quality issues, not for
# explainable events (holidays, campaigns). Those should be modelled.
prophet_df.loc[prophet_df['is_outlier'], 'y'] = np.nan

# Clean up helper columns
prophet_df = prophet_df.drop(columns=['dow', 'is_outlier'])


### 2.3 Build the Holiday DataFrame

Prophet's holiday framework models each holiday as a separate effect with configurable windows before and after:


In [ ]:
# ──────────────────────────────────────────────────
# Irish Bank Holidays
# ──────────────────────────────────────────────────

# Define Irish bank holidays for the full date range
# Source: https://www.gov.ie/en/publication/public-holidays/
def get_irish_bank_holidays(start_year, end_year):
    """Generate Irish public holidays for the given year range."""
    holidays = []

    for year in range(start_year, end_year + 1):
        # Fixed-date holidays
        holidays.append({'ds': f'{year}-01-01', 'holiday': "New Year's Day"})
        holidays.append({'ds': f'{year}-02-01', 'holiday': "St Brigid's Day"})
        holidays.append({'ds': f'{year}-03-17', 'holiday': "St Patrick's Day"})
        holidays.append({'ds': f'{year}-05-06', 'holiday': 'May Bank Holiday'})  # First Mon in May (approximate)
        holidays.append({'ds': f'{year}-06-03', 'holiday': 'June Bank Holiday'})  # First Mon in June (approximate)
        holidays.append({'ds': f'{year}-08-05', 'holiday': 'August Bank Holiday'})  # First Mon in August (approximate)
        holidays.append({'ds': f'{year}-10-28', 'holiday': 'October Bank Holiday'})  # Last Mon in October (approximate)
        holidays.append({'ds': f'{year}-12-25', 'holiday': 'Christmas Day'})
        holidays.append({'ds': f'{year}-12-26', 'holiday': "St Stephen's Day"})

        # Easter (variable) — use a library for accurate dates
        # pip install dateutil
        from dateutil.easter import easter
        easter_date = easter(year)
        holidays.append({'ds': str(easter_date - pd.Timedelta(days=2)), 'holiday': 'Good Friday'})
        holidays.append({'ds': str(easter_date + pd.Timedelta(days=1)), 'holiday': 'Easter Monday'})

    return pd.DataFrame(holidays)

holidays_df = get_irish_bank_holidays(2022, 2027)
holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])

# ──────────────────────────────────────────────────
# Configure holiday windows
# ──────────────────────────────────────────────────
# lower_window = days BEFORE the holiday that are affected
# upper_window = days AFTER the holiday that are affected

# The day AFTER a bank holiday is often the highest-volume day of the month
# because callers who couldn't get through on the holiday call the next morning.
# This rebound effect is the single most important holiday feature.

# Standard bank holidays: quiet on the day, spike the day after
standard_holidays = holidays_df[~holidays_df['holiday'].isin(['Christmas Day', "St Stephen's Day"])].copy()
standard_holidays['lower_window'] = 0   # No effect before
standard_holidays['upper_window'] = 1   # Rebound the day after

# Christmas period: extended closure with multi-day rebound
christmas = holidays_df[holidays_df['holiday'].isin(['Christmas Day', "St Stephen's Day"])].copy()
christmas_day = christmas[christmas['holiday'] == 'Christmas Day'].copy()
christmas_day['lower_window'] = -1  # Christmas Eve often sees reduced volume
christmas_day['upper_window'] = 4   # Takes several days to clear the backlog

# Combine
holidays_final = pd.concat([standard_holidays, christmas_day], ignore_index=True)

print(f'Holiday events defined: {len(holidays_final)}')
print(holidays_final.head(15))


### 2.4 Build Custom Regressors

These capture effects that Prophet's seasonality and holidays cannot:


In [ ]:
# ──────────────────────────────────────────────────
# Banking-specific regressors
# ──────────────────────────────────────────────────

def add_banking_regressors(df):
    """Add domain-specific features to the Prophet DataFrame."""
    df = df.copy()

    # Direct debit processing dates
    # The 1st and 15th of each month generate failed-payment calls
    df['is_direct_debit'] = df['ds'].dt.day.isin([1, 2, 15, 16]).astype(float)

    # Payday window (most Irish salaries paid last working day of month or 15th)
    df['is_payday_window'] = ((df['ds'].dt.day >= 25) | (df['ds'].dt.day <= 3)).astype(float)

    # Month-end processing (statement queries, fee disputes)
    df['is_month_end'] = (df['ds'].dt.day >= 28).astype(float)

    # January effect (New Year financial resolutions, tax prep)
    df['is_january'] = (df['ds'].dt.month == 1).astype(float)

    # Tax deadline periods (Irish self-assessment: Oct 31, preliminary tax: Nov)
    df['is_tax_season'] = df['ds'].dt.month.isin([10, 11]).astype(float)

    # Weekend flag (if centre operates 7 days but with reduced weekend hours)
    df['is_weekend'] = (df['ds'].dt.dayofweek >= 5).astype(float)

    return df

prophet_df = add_banking_regressors(prophet_df)

# Verify
print('Regressor columns:')
print([col for col in prophet_df.columns if col not in ['ds', 'y']])
print(f'\nSample:')
print(prophet_df[['ds', 'y', 'is_direct_debit', 'is_payday_window', 'is_weekend']].tail(10))


### 2.5 Train/Test Split


In [ ]:
# ──────────────────────────────────────────────────
# Hold out the most recent 8 weeks for evaluation
# This gives 8 non-overlapping 7-day forecast windows
# ──────────────────────────────────────────────────
FORECAST_HORIZON = 7
N_TEST_WEEKS = 8
TEST_DAYS = N_TEST_WEEKS * FORECAST_HORIZON

cutoff_date = prophet_df['ds'].max() - pd.Timedelta(days=TEST_DAYS - 1)
train = prophet_df[prophet_df['ds'] < cutoff_date].copy()
test = prophet_df[prophet_df['ds'] >= cutoff_date].copy()

print(f'Train: {train["ds"].min().date()} to {train["ds"].max().date()} ({len(train)} days)')
print(f'Test : {test["ds"].min().date()} to {test["ds"].max().date()} ({len(test)} days)')
print(f'Test weeks: {N_TEST_WEEKS}')


---


## Stage 3: MODEL

### 3.1 Baseline — Seasonal Naïve

Always establish the baseline before fitting Prophet. This is the benchmark that Prophet must beat to be worth deploying:


In [ ]:
# ──────────────────────────────────────────────────
# Seasonal naïve: same day last week
# ──────────────────────────────────────────────────
test_with_baseline = test.copy()
test_with_baseline['naive_lag7'] = prophet_df.set_index('ds')['y'].shift(7).loc[test['ds']].values

# 4-week same-day average
for i, row in test_with_baseline.iterrows():
    same_dow_dates = [row['ds'] - pd.Timedelta(weeks=w) for w in range(1, 5)]
    same_dow_vals = prophet_df[prophet_df['ds'].isin(same_dow_dates)]['y']
    test_with_baseline.loc[i, 'naive_avg4w'] = same_dow_vals.mean()

baseline_mae = np.abs(test_with_baseline['y'] - test_with_baseline['naive_lag7']).mean()
baseline_mape = (np.abs(test_with_baseline['y'] - test_with_baseline['naive_lag7']) / test_with_baseline['y']).mean() * 100
print(f'Seasonal Naïve Baseline — MAE: {baseline_mae:.0f}, MAPE: {baseline_mape:.1f}%')

avg4w_mae = np.abs(test_with_baseline['y'] - test_with_baseline['naive_avg4w']).mean()
avg4w_mape = (np.abs(test_with_baseline['y'] - test_with_baseline['naive_avg4w']) / test_with_baseline['y']).mean() * 100
print(f'4-Week Avg Baseline    — MAE: {avg4w_mae:.0f}, MAPE: {avg4w_mape:.1f}%')


### 3.2 Prophet — Configuration and Fitting


In [ ]:
# ──────────────────────────────────────────────────
# Configure the Prophet model
# ──────────────────────────────────────────────────

model = Prophet(
    # ── Trend ──
    growth='linear',                  # Linear trend (use 'logistic' if volume has a ceiling)
    changepoint_prior_scale=0.05,     # Controls trend flexibility
                                      # Lower = smoother trend, less overfitting
                                      # Higher = more responsive to level shifts
                                      # Default 0.05 is good; tune between 0.001 and 0.5
    changepoint_range=0.9,            # Look for changepoints in first 90% of data
                                      # (default 0.8 — extending to 0.9 helps if
                                      # recent trend shifts are important)
    n_changepoints=25,                # Number of potential changepoint locations

    # ── Seasonality ──
    yearly_seasonality=True,          # Annual cycle (Fourier order auto-selected)
    weekly_seasonality=True,          # Weekly cycle (default Fourier order = 3)
    daily_seasonality=False,          # Not applicable for daily-level data

    seasonality_mode='additive',      # IMPORTANT DECISION
                                      # Use 'additive' if seasonal swings are constant
                                      # in absolute terms regardless of the level.
                                      # Use 'multiplicative' if swings grow proportionally.
                                      # For call centre data, 'additive' is usually correct.
                                      # Validate by checking STL residuals (Stage 1).

    seasonality_prior_scale=10.0,     # Regularisation on seasonal components
                                      # Lower = smoother seasonality, less overfitting
                                      # Higher = allows sharper seasonal patterns
                                      # Default 10 is usually fine for weekly/yearly

    # ── Holidays ──
    holidays=holidays_final,
    holidays_prior_scale=10.0,        # Regularisation on holiday effects
                                      # Lower = shrink holiday effects toward zero
                                      # Higher = allow large holiday spikes/drops
                                      # For call centres, holidays have BIG effects,
                                      # so keep this at 10+ or even increase to 20

    # ── Uncertainty ──
    interval_width=0.80,              # 80% prediction intervals
    mcmc_samples=0,                   # 0 = MAP estimation (fast)
                                      # 300+ = full Bayesian (slow but better intervals)
)

# ── Add custom regressors ──
# Each regressor gets its own regularisation prior
model.add_regressor('is_direct_debit', prior_scale=10.0, mode='additive')
model.add_regressor('is_payday_window', prior_scale=5.0, mode='additive')
model.add_regressor('is_month_end', prior_scale=5.0, mode='additive')
model.add_regressor('is_january', prior_scale=5.0, mode='additive')
model.add_regressor('is_tax_season', prior_scale=5.0, mode='additive')
model.add_regressor('is_weekend', prior_scale=15.0, mode='additive')  # Strong known effect

# ── Optional: add country-level holidays as a catch-all ──
# This adds Prophet's built-in Irish holiday calendar ON TOP of your custom holidays.
# It can catch holidays you missed, but may also double-count.
# Test with and without to see which performs better.
# model.add_country_holidays(country_name='IE')

# ──────────────────────────────────────────────────
# Fit the model
# ──────────────────────────────────────────────────
model.fit(train)
print('Model fitted successfully.')


### 3.3 Understanding What Prophet Learned


In [ ]:
# ──────────────────────────────────────────────────
# Inspect the components
# ──────────────────────────────────────────────────

# Generate in-sample predictions
train_forecast = model.predict(train)

# Plot all components
fig = model.plot_components(train_forecast)
fig.set_size_inches(14, 16)
plt.suptitle('Prophet Components', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()


**Reading the component plots:**

- **Trend panel:** Shows the piecewise linear trend with changepoints. Are the changepoints at sensible dates? Do they correspond to known events (branch closures, system changes, COVID)? If the trend looks too wiggly, reduce `changepoint_prior_scale`.

- **Weekly panel:** Shows the day-of-week effect. Monday should be highest, weekend lowest. Does the shape match your box plots from Stage 1? If not, increase `weekly_seasonality` Fourier order.

- **Yearly panel:** Shows the annual cycle. January and October (tax season) should be high; August should be low. If this looks too smooth and misses known seasonal features, increase the yearly Fourier order.

- **Holiday panels:** Shows the estimated effect of each holiday. Christmas should show a large negative effect (closure) followed by positive rebounds.

- **Regressor panels:** Shows the estimated effect of each custom regressor. `is_direct_debit` should be positive (more calls on DD days). `is_weekend` should be strongly negative.


In [ ]:
# ──────────────────────────────────────────────────
# Inspect changepoints
# ──────────────────────────────────────────────────
from prophet.plot import add_changepoints_to_plot

fig = model.plot(train_forecast)
add_changepoints_to_plot(fig.gca(), model, train_forecast)
fig.set_size_inches(18, 6)
plt.title('Fitted Values with Changepoints (red dashed lines)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print changepoint dates and magnitudes
changepoints = model.changepoints
deltas = model.params['delta'].mean(axis=0)
significant_cps = pd.DataFrame({
    'date': changepoints,
    'delta': deltas
}).sort_values('delta', key=abs, ascending=False)

print('Top 10 changepoints by magnitude:')
print(significant_cps.head(10).to_string(index=False))


### 3.4 Generate the 7-Day Forecast


In [ ]:
# ──────────────────────────────────────────────────
# Create the future DataFrame
# ──────────────────────────────────────────────────

# Prophet's make_future_dataframe generates dates, but we need to
# populate the regressor columns for the forecast period

future = model.make_future_dataframe(periods=FORECAST_HORIZON, freq='D')

# Add the same regressors to the future DataFrame
# CRITICAL: These must be populated with KNOWN values for the future dates.
# Calendar-based features are deterministic — you know what day of the week
# next Tuesday is, and whether the 1st falls next week.
future = add_banking_regressors(future)

# Sanity check: no NaN in regressors for future dates
future_only = future[future['ds'] > train['ds'].max()]
print('Future dates:')
print(future_only[['ds', 'is_direct_debit', 'is_payday_window', 'is_weekend']])
assert future_only[['is_direct_debit', 'is_payday_window', 'is_month_end',
                     'is_january', 'is_tax_season', 'is_weekend']].isna().sum().sum() == 0, \
    'ERROR: NaN values in future regressors!'

# ──────────────────────────────────────────────────
# Generate forecast
# ──────────────────────────────────────────────────
forecast = model.predict(future)

# Extract the 7-day forecast
forecast_7d = forecast[forecast['ds'] > train['ds'].max()].copy()
forecast_7d = forecast_7d[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

print('\n7-Day Forecast:')
print('─' * 65)
print(f'{"Date":<14} {"Day":<12} {"Forecast":>10} {"Lower 80%":>12} {"Upper 80%":>12}')
print('─' * 65)
for _, row in forecast_7d.iterrows():
    day_name = row['ds'].strftime('%A')
    print(f'{row["ds"].strftime("%Y-%m-%d"):<14} {day_name:<12} '
          f'{row["yhat"]:>10,.0f} {row["yhat_lower"]:>12,.0f} {row["yhat_upper"]:>12,.0f}')
print('─' * 65)
print(f'{"Weekly Total":<26} {forecast_7d["yhat"].sum():>10,.0f} '
      f'{forecast_7d["yhat_lower"].sum():>12,.0f} {forecast_7d["yhat_upper"].sum():>12,.0f}')


### 3.5 Visualise the Forecast


In [ ]:
# ──────────────────────────────────────────────────
# Plot: recent history + forecast
# ──────────────────────────────────────────────────
lookback_days = 60
recent = forecast[forecast['ds'] >= (train['ds'].max() - pd.Timedelta(days=lookback_days))]

fig, ax = plt.subplots(figsize=(18, 6))

# Historical actuals
hist_mask = recent['ds'] <= train['ds'].max()
ax.plot(recent.loc[hist_mask, 'ds'], train.set_index('ds').loc[recent.loc[hist_mask, 'ds'], 'y'],
        color='#2c3e50', linewidth=1, label='Actual', marker='o', markersize=2)

# Prophet fitted values (recent history)
ax.plot(recent.loc[hist_mask, 'ds'], recent.loc[hist_mask, 'yhat'],
        color='#3498db', linewidth=1, alpha=0.5, label='Prophet Fitted')

# Forecast
forecast_mask = recent['ds'] > train['ds'].max()
ax.plot(recent.loc[forecast_mask, 'ds'], recent.loc[forecast_mask, 'yhat'],
        color='#e74c3c', linewidth=2, marker='o', markersize=6, label='7-Day Forecast')
ax.fill_between(recent.loc[forecast_mask, 'ds'],
                recent.loc[forecast_mask, 'yhat_lower'],
                recent.loc[forecast_mask, 'yhat_upper'],
                color='#e74c3c', alpha=0.15, label='80% Prediction Interval')

# Cutoff line
ax.axvline(x=train['ds'].max(), color='grey', linestyle='--', linewidth=1, alpha=0.7, label='Forecast Origin')

ax.set_title('Daily Call Volume: Recent History + 7-Day Prophet Forecast', fontsize=14, fontweight='bold')
ax.set_ylabel('Calls')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()


---


## Stage 4: DIAGNOSE

### 4.1 In-Sample Residual Analysis


In [ ]:
# ──────────────────────────────────────────────────
# Compute residuals
# ──────────────────────────────────────────────────
train_with_fit = train.merge(
    train_forecast[['ds', 'yhat']], on='ds', how='left'
)
train_with_fit['residual'] = train_with_fit['y'] - train_with_fit['yhat']
train_with_fit['pct_error'] = train_with_fit['residual'] / train_with_fit['y'] * 100

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Residuals over time
axes[0, 0].plot(train_with_fit['ds'], train_with_fit['residual'], linewidth=0.4, color='#2c3e50')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=0.8)
axes[0, 0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Residual (calls)')

# 2. Histogram
axes[0, 1].hist(train_with_fit['residual'].dropna(), bins=50, color='#3498db', edgecolor='white')
axes[0, 1].axvline(x=0, color='red', linestyle='--')
axes[0, 1].set_title('Residual Distribution', fontsize=12, fontweight='bold')

# 3. QQ plot
from scipy import stats
stats.probplot(train_with_fit['residual'].dropna(), plot=axes[0, 2])
axes[0, 2].set_title('QQ Plot', fontsize=12, fontweight='bold')

# 4. ACF of residuals
plot_acf(train_with_fit['residual'].dropna(), lags=35, ax=axes[1, 0])
axes[1, 0].set_title('ACF of Residuals', fontsize=12, fontweight='bold')

# 5. Residuals by day of week
train_with_fit['dow'] = train_with_fit['ds'].dt.day_name()
sns.boxplot(data=train_with_fit, x='dow', y='residual',
            order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            ax=axes[1, 1])
axes[1, 1].axhline(y=0, color='red', linestyle='--')
axes[1, 1].set_title('Residuals by Day of Week', fontsize=12, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Residuals vs fitted (check for heteroscedasticity)
axes[1, 2].scatter(train_with_fit['yhat'], train_with_fit['residual'],
                   alpha=0.2, s=5, color='#2c3e50')
axes[1, 2].axhline(y=0, color='red', linestyle='--')
axes[1, 2].set_title('Residuals vs Fitted', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Fitted Value')
axes[1, 2].set_ylabel('Residual')

plt.suptitle('Prophet Residual Diagnostics', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()


**Diagnostic checklist — what to look for and what to do about it:**

| Observation | Diagnosis | Action |
|---|---|---|
| Residuals have significant ACF at lag 7 | Prophet's weekly seasonality is insufficient | Increase `weekly_seasonality` Fourier order: `model = Prophet(weekly_seasonality=10)` instead of default 3 |
| Residuals have significant ACF at lag 1 | Short-term autocorrelation not captured | Prophet fundamentally does not model autocorrelation. Consider post-processing residuals with an AR(1) model, or switch to a hybrid approach |
| Residual box plots show Monday bias (positive) | Prophet underpredicts Mondays | Add a specific `is_monday` regressor, or increase Fourier order for weekly seasonality |
| Funnel shape in residuals-vs-fitted | Heteroscedasticity | Switch to `seasonality_mode='multiplicative'` or apply log transform to `y` |
| Residuals spike on specific dates | Missed holiday or event | Add the event to the holidays DataFrame |
| Residual variance increases over time | Model drift | Retrain more frequently or reduce `changepoint_range` |

### 4.2 Ljung-Box Test on Residuals


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

lb = acorr_ljungbox(train_with_fit['residual'].dropna(), lags=[7, 14, 21, 28], return_df=True)
print('Ljung-Box Test (H₀: no autocorrelation in residuals):')
print(lb)
print()
for lag, row in lb.iterrows():
    status = '✅ No autocorrelation' if row['lb_pvalue'] > 0.05 else '❌ Autocorrelation remains'
    print(f'  Lag {lag:2d}: p={row["lb_pvalue"]:.4f}  {status}')


**If the Ljung-Box test fails (common with Prophet):** Prophet treats errors as independent, which means it does not model the autoregressive structure that remains in residuals. This is Prophet's main theoretical limitation. Options:

1. **Accept it** — for a 7-day horizon, the impact on forecast accuracy is often modest
2. **Post-process with an ARIMA on residuals** — fit an AR model to Prophet's residuals and add the AR forecast to Prophet's forecast (hybrid approach)
3. **Switch to a model that captures autocorrelation** — XGBoost with lag features, or SARIMA

### 4.3 Prophet Cross-Validation

Prophet has a built-in cross-validation framework that simulates the rolling-origin evaluation you would do in production:


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# ──────────────────────────────────────────────────
# Cross-validation setup
# ──────────────────────────────────────────────────
# initial  : minimum training data before first forecast
# period   : spacing between forecast origins (every 7 days = weekly)
# horizon  : how far ahead to forecast (7 days)

cv_results = cross_validation(
    model,
    initial='730 days',   # Use at least 2 years of training data
    period='7 days',      # Make a new forecast every week
    horizon='7 days',     # Forecast 7 days ahead each time
    parallel='processes'  # Speed up with multiprocessing
)

print(f'Cross-validation produced {len(cv_results)} forecast-actual pairs')
print(f'Across {cv_results["cutoff"].nunique()} forecast origins')
print(cv_results.head(15))


In [ ]:
# ──────────────────────────────────────────────────
# Performance metrics
# ──────────────────────────────────────────────────
metrics = performance_metrics(cv_results, rolling_window=1)
print(metrics[['horizon', 'mae', 'mape', 'rmse', 'coverage']].to_string(index=False))

# Overall performance
print(f'\nOverall CV Performance:')
print(f'  MAE     : {metrics["mae"].mean():.0f} calls')
print(f'  MAPE    : {metrics["mape"].mean() * 100:.1f}%')
print(f'  RMSE    : {metrics["rmse"].mean():.0f} calls')
print(f'  Coverage: {metrics["coverage"].mean() * 100:.1f}% (target: 80%)')


In [ ]:
# ──────────────────────────────────────────────────
# Performance by forecast horizon
# ──────────────────────────────────────────────────
fig = plot_cross_validation_metric(cv_results, metric='mape')
fig.set_size_inches(14, 5)
plt.title('MAPE by Forecast Horizon (days ahead)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# ──────────────────────────────────────────────────
# Performance by day of week
# ──────────────────────────────────────────────────
cv_results['dow'] = cv_results['ds'].dt.day_name()
cv_results['abs_pct_error'] = np.abs((cv_results['y'] - cv_results['yhat']) / cv_results['y'])

dow_perf = cv_results.groupby('dow')['abs_pct_error'].agg(['mean', 'median', 'std']).reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
) * 100

print('\nMAPE by Day of Week:')
print(dow_perf.round(1).rename(columns={'mean': 'Mean MAPE%', 'median': 'Median MAPE%', 'std': 'Std%'}))


In [ ]:
# ──────────────────────────────────────────────────
# Performance over time (detect drift)
# ──────────────────────────────────────────────────
cv_by_cutoff = cv_results.groupby('cutoff').agg(
    mape=('abs_pct_error', 'mean'),
    mae=('y', lambda x: np.abs(cv_results.loc[x.index, 'y'] - cv_results.loc[x.index, 'yhat']).mean())
).reset_index()

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(cv_by_cutoff['cutoff'], cv_by_cutoff['mape'] * 100, marker='o', markersize=3, linewidth=0.8)
ax.axhline(y=10, color='orange', linestyle='--', label='10% MAPE threshold')
ax.set_title('Cross-Validation MAPE Over Time (each point = one 7-day forecast)', fontsize=14, fontweight='bold')
ax.set_ylabel('MAPE (%)')
ax.legend()
plt.tight_layout()
plt.show()


### 4.4 Prediction Interval Calibration

A forecast is only as useful as its uncertainty estimate. If the model says "80% interval", roughly 80% of actuals should fall within it:


In [ ]:
# Check empirical coverage
cv_results['within_interval'] = (
    (cv_results['y'] >= cv_results['yhat_lower']) &
    (cv_results['y'] <= cv_results['yhat_upper'])
)

overall_coverage = cv_results['within_interval'].mean() * 100
print(f'Empirical coverage: {overall_coverage:.1f}% (target: {model.interval_width * 100:.0f}%)')

# Coverage by day of week
dow_coverage = cv_results.groupby('dow')['within_interval'].mean().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
) * 100
print('\nCoverage by Day of Week:')
for day, cov in dow_coverage.items():
    status = '✅' if abs(cov - 80) < 10 else '⚠️'
    print(f'  {day:12s}: {cov:.1f}%  {status}')


**If coverage is too low (< 75%):** Intervals are too narrow. Increase `mcmc_samples` (300+) for better uncertainty estimation, or widen the `interval_width`. If coverage is consistently low on Mondays, the model is underestimating Monday variance.

**If coverage is too high (> 90%):** Intervals are too wide, which reduces their operational utility. The staffing model will schedule too conservatively. Consider reducing `interval_width` or investigating whether outlier removal in preprocessing was too aggressive.

---


## Stage 5: DEPLOY AND MONITOR

### 5.1 Hyperparameter Tuning

Before deploying, systematically tune the key parameters:


In [ ]:
# ──────────────────────────────────────────────────
# Grid search over Prophet's most impactful parameters
# ──────────────────────────────────────────────────
from itertools import product

param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [1.0, 5.0, 10.0],
    'holidays_prior_scale': [5.0, 10.0, 20.0],
    'seasonality_mode': ['additive', 'multiplicative'],
}

# Generate all combinations
all_params = [dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())]
print(f'Testing {len(all_params)} parameter combinations...')

results = []
for i, params in enumerate(all_params):
    m = Prophet(
        growth='linear',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        holidays=holidays_final,
        **params
    )

    # Add regressors
    for reg in ['is_direct_debit', 'is_payday_window', 'is_month_end',
                'is_january', 'is_tax_season', 'is_weekend']:
        m.add_regressor(reg, mode=params['seasonality_mode'])

    m.fit(train)

    cv = cross_validation(m, initial='730 days', period='7 days', horizon='7 days', parallel='processes')
    metrics = performance_metrics(cv, rolling_window=1)

    result = {**params,
              'mae': metrics['mae'].mean(),
              'mape': metrics['mape'].mean(),
              'rmse': metrics['rmse'].mean(),
              'coverage': metrics['coverage'].mean()}
    results.append(result)

    if (i + 1) % 10 == 0:
        print(f'  Completed {i + 1}/{len(all_params)}')

results_df = pd.DataFrame(results).sort_values('mape')
print('\nTop 5 parameter combinations by MAPE:')
print(results_df.head().to_string(index=False))

best_params = results_df.iloc[0].to_dict()
print(f'\nBest: MAPE={best_params["mape"]*100:.1f}%, '
      f'changepoint_prior={best_params["changepoint_prior_scale"]}, '
      f'seasonality_mode={best_params["seasonality_mode"]}')


### 5.2 Production Forecast Script


In [ ]:
# ──────────────────────────────────────────────────
# production_forecast.py
# Run every Monday morning at 06:45
# ──────────────────────────────────────────────────

import pandas as pd
import numpy as np
from prophet import Prophet
from datetime import datetime, timedelta
import json
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

FORECAST_HORIZON = 7
BEST_PARAMS = {
    'changepoint_prior_scale': 0.05,
    'seasonality_prior_scale': 10.0,
    'holidays_prior_scale': 10.0,
    'seasonality_mode': 'additive',
}

def load_latest_data():
    """Pull latest call volume data from the ACD/CRM system."""
    # In production, this queries your data warehouse
    df = pd.read_csv('daily_call_volume.csv', parse_dates=['date'])
    df = df.rename(columns={'date': 'ds', 'calls': 'y'})
    return df

def data_quality_checks(df):
    """Validate the data before modelling."""
    checks = {}

    # Check for missing recent days
    expected_last_date = pd.Timestamp.today().normalize() - pd.Timedelta(days=1)
    actual_last_date = df['ds'].max()
    checks['last_date'] = actual_last_date
    checks['data_current'] = actual_last_date >= expected_last_date - pd.Timedelta(days=2)

    # Check for implausible values
    recent = df[df['ds'] >= actual_last_date - pd.Timedelta(days=7)]
    checks['recent_mean'] = recent['y'].mean()
    checks['any_negative'] = (recent['y'] < 0).any()
    checks['any_extreme'] = (recent['y'] > df['y'].quantile(0.999) * 2).any()

    # Check for missing values
    checks['missing_last_7d'] = recent['y'].isna().sum()

    for check, value in checks.items():
        logger.info(f'  {check}: {value}')

    if checks['any_negative'] or checks['any_extreme'] or not checks['data_current']:
        raise ValueError(f'Data quality check FAILED: {checks}')

    return checks

def build_and_forecast(df, holidays_df, horizon=FORECAST_HORIZON):
    """Fit Prophet and generate forecast."""
    model = Prophet(
        growth='linear',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        holidays=holidays_df,
        interval_width=0.80,
        **BEST_PARAMS
    )

    for reg in ['is_direct_debit', 'is_payday_window', 'is_month_end',
                'is_january', 'is_tax_season', 'is_weekend']:
        model.add_regressor(reg, mode=BEST_PARAMS['seasonality_mode'])

    df = add_banking_regressors(df)
    model.fit(df)

    future = model.make_future_dataframe(periods=horizon, freq='D')
    future = add_banking_regressors(future)
    forecast = model.predict(future)

    # Extract forecast period only
    result = forecast[forecast['ds'] > df['ds'].max()][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()

    # Floor at zero (call volume cannot be negative)
    result['yhat'] = result['yhat'].clip(lower=0)
    result['yhat_lower'] = result['yhat_lower'].clip(lower=0)

    return model, result

def sanity_check_forecast(forecast_df, df):
    """Verify the forecast is reasonable before publishing."""
    historical_mean = df['y'].tail(90).mean()
    forecast_mean = forecast_df['yhat'].mean()

    checks = {
        'forecast_mean': forecast_mean,
        'historical_90d_mean': historical_mean,
        'ratio': forecast_mean / historical_mean,
        'any_negative': (forecast_df['yhat'] < 0).any(),
        'all_same': forecast_df['yhat'].std() < 1,
    }

    # Forecast should be within 50% of recent average
    if not (0.5 < checks['ratio'] < 1.5):
        raise ValueError(f'Forecast seems implausible: ratio={checks["ratio"]:.2f}')

    if checks['all_same']:
        raise ValueError('Forecast is suspiciously flat')

    logger.info(f'  Forecast sanity check PASSED: ratio={checks["ratio"]:.2f}')
    return checks

def generate_staffing_inputs(forecast_df, aht_seconds=300, service_level=0.80,
                             target_asa=20, shrinkage=0.30):
    """Convert daily forecast to staffing requirements via Erlang C."""
    from math import factorial

    def erlang_c_agents(calls_per_day, aht, intervals_per_day=34):
        """Calculate agents needed per interval assuming uniform distribution.
           In production, use the actual intraday arrival profile."""
        calls_per_interval = calls_per_day / intervals_per_day
        interval_seconds = (12 * 3600) / intervals_per_day  # ~12 operating hours
        traffic = (calls_per_interval * aht) / interval_seconds

        for n in range(max(1, int(traffic) + 1), int(traffic) + 200):
            sum_terms = sum((traffic ** k) / factorial(k) for k in range(n))
            last_term = (traffic ** n) / factorial(n) * (n / (n - traffic))
            pw = last_term / (sum_terms + last_term)
            sl = 1 - pw * np.exp(-(n - traffic) * target_asa / aht)
            if sl >= service_level:
                return n
        return n

    staffing = forecast_df.copy()
    staffing['raw_agents'] = staffing['yhat'].apply(lambda x: erlang_c_agents(x, aht_seconds))
    staffing['scheduled_agents'] = np.ceil(staffing['raw_agents'] / (1 - shrinkage)).astype(int)
    staffing['day'] = staffing['ds'].dt.day_name()

    return staffing[['ds', 'day', 'yhat', 'yhat_lower', 'yhat_upper', 'raw_agents', 'scheduled_agents']]

# ──────────────────────────────────────────────────
# Main execution
# ──────────────────────────────────────────────────
if __name__ == '__main__':
    logger.info('=== WEEKLY FORECAST PIPELINE STARTED ===')

    # Step 1: Load data
    logger.info('Loading data...')
    df = load_latest_data()

    # Step 2: Quality checks
    logger.info('Running data quality checks...')
    data_quality_checks(df)

    # Step 3: Build forecast
    logger.info('Fitting Prophet and generating 7-day forecast...')
    holidays_df = get_irish_bank_holidays(df['ds'].dt.year.min(), df['ds'].dt.year.max() + 1)
    model, forecast_7d = build_and_forecast(df, holidays_df)

    # Step 4: Sanity check
    logger.info('Sanity checking forecast...')
    sanity_check_forecast(forecast_7d, df)

    # Step 5: Generate staffing plan
    logger.info('Generating staffing requirements...')
    staffing = generate_staffing_inputs(forecast_7d)

    print('\n' + '=' * 80)
    print('7-DAY CALL VOLUME FORECAST AND STAFFING PLAN')
    print('=' * 80)
    print(f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M")}')
    print(f'Forecast period: {forecast_7d["ds"].min().strftime("%Y-%m-%d")} to '
          f'{forecast_7d["ds"].max().strftime("%Y-%m-%d")}')
    print('-' * 80)
    print(f'{"Date":<12} {"Day":<10} {"Forecast":>9} {"Low 80%":>9} {"High 80%":>9} '
          f'{"Agents":>8} {"Scheduled":>10}')
    print('-' * 80)
    for _, row in staffing.iterrows():
        print(f'{row["ds"].strftime("%Y-%m-%d"):<12} {row["day"]:<10} '
              f'{row["yhat"]:>9,.0f} {row["yhat_lower"]:>9,.0f} {row["yhat_upper"]:>9,.0f} '
              f'{row["raw_agents"]:>8d} {row["scheduled_agents"]:>10d}')
    print('-' * 80)
    print(f'{"TOTAL":<22} {staffing["yhat"].sum():>9,.0f} {staffing["yhat_lower"].sum():>9,.0f} '
          f'{staffing["yhat_upper"].sum():>9,.0f}')
    print('=' * 80)

    # Step 6: Export
    staffing.to_csv(f'forecast_{datetime.now().strftime("%Y%m%d")}.csv', index=False)
    logger.info(f'Forecast exported to forecast_{datetime.now().strftime("%Y%m%d")}.csv')
    logger.info('=== PIPELINE COMPLETE ===')


### 5.3 Weekly Monitoring


In [ ]:
# ──────────────────────────────────────────────────
# monitor_forecast.py
# Run every Monday AFTER actuals for last week are available
# ──────────────────────────────────────────────────

def weekly_accuracy_report(actuals_df, forecast_df):
    """
    Compare last week's forecast against actuals.
    
    Parameters:
        actuals_df: DataFrame with 'ds' and 'y' (actual call volumes)
        forecast_df: DataFrame with 'ds', 'yhat', 'yhat_lower', 'yhat_upper'
    """
    merged = actuals_df.merge(forecast_df, on='ds', how='inner')
    merged['error'] = merged['y'] - merged['yhat']
    merged['abs_error'] = merged['error'].abs()
    merged['pct_error'] = merged['error'] / merged['y'] * 100
    merged['abs_pct_error'] = merged['pct_error'].abs()
    merged['within_interval'] = (merged['y'] >= merged['yhat_lower']) & (merged['y'] <= merged['yhat_upper'])
    merged['day'] = merged['ds'].dt.day_name()

    # Summary metrics
    report = {
        'week_ending': merged['ds'].max(),
        'mae': merged['abs_error'].mean(),
        'mape': merged['abs_pct_error'].mean(),
        'rmse': np.sqrt((merged['error'] ** 2).mean()),
        'bias': merged['error'].mean(),
        'coverage': merged['within_interval'].mean() * 100,
        'worst_day': merged.loc[merged['abs_pct_error'].idxmax(), 'day'],
        'worst_day_error': merged['abs_pct_error'].max(),
    }

    # Print report
    print('\n' + '=' * 70)
    print(f'FORECAST ACCURACY REPORT — Week Ending {report["week_ending"].strftime("%Y-%m-%d")}')
    print('=' * 70)
    print(f'  MAE       : {report["mae"]:>8,.0f} calls')
    print(f'  MAPE      : {report["mape"]:>8.1f}%')
    print(f'  RMSE      : {report["rmse"]:>8,.0f} calls')
    print(f'  Bias      : {report["bias"]:>+8,.0f} calls (positive = underprediction)')
    print(f'  Coverage  : {report["coverage"]:>8.1f}% (target: 80%)')
    print(f'  Worst day : {report["worst_day"]} ({report["worst_day_error"]:.1f}% error)')
    print()

    # Day-by-day breakdown
    print(f'{"Date":<12} {"Day":<10} {"Actual":>8} {"Forecast":>9} {"Error":>8} {"%Error":>8} {"In CI?":>7}')
    print('-' * 70)
    for _, row in merged.iterrows():
        ci_flag = '✅' if row['within_interval'] else '❌'
        print(f'{row["ds"].strftime("%Y-%m-%d"):<12} {row["day"]:<10} '
              f'{row["y"]:>8,.0f} {row["yhat"]:>9,.0f} {row["error"]:>+8,.0f} '
              f'{row["pct_error"]:>+7.1f}% {ci_flag:>7}')

    # Alerts
    print()
    if report['mape'] > 15:
        print('🚨 ALERT: MAPE exceeds 15% — investigate and consider retraining')
    elif report['mape'] > 10:
        print('⚠️  WARNING: MAPE above 10% — monitor closely next week')
    else:
        print('✅ Performance within acceptable range')

    if abs(report['bias']) > 0.08 * merged['y'].mean():
        direction = 'underpredicting' if report['bias'] > 0 else 'overpredicting'
        print(f'⚠️  BIAS ALERT: Model is systematically {direction} '
              f'by {abs(report["bias"]):.0f} calls/day')

    if report['coverage'] < 65:
        print('⚠️  COVERAGE ALERT: Prediction intervals are too narrow — '
              f'{report["coverage"]:.0f}% vs 80% target')

    return report


### 5.4 Retraining Schedule and Drift Detection


In [ ]:
# ──────────────────────────────────────────────────
# Retraining decision logic
# ──────────────────────────────────────────────────

def should_retrain(performance_history, weeks_since_retrain):
    """
    Decide whether to retrain the model based on recent performance.

    Returns: (should_retrain: bool, reason: str)
    """
    if len(performance_history) < 4:
        return False, 'Insufficient history for drift detection'

    recent = performance_history.tail(4)

    # Trigger 1: Rolling MAPE above threshold
    rolling_mape = recent['mape'].mean()
    if rolling_mape > 12:
        return True, f'Rolling 4-week MAPE = {rolling_mape:.1f}% (threshold: 12%)'

    # Trigger 2: Systematic bias
    rolling_bias = recent['bias'].mean()
    avg_volume = recent['mae'].mean() / (recent['mape'].mean() / 100)  # Approximate
    if abs(rolling_bias) > 0.08 * avg_volume:
        return True, f'Systematic bias = {rolling_bias:+.0f} calls/day'

    # Trigger 3: Time-based (retrain at least monthly)
    if weeks_since_retrain >= 4:
        return True, f'{weeks_since_retrain} weeks since last retrain (threshold: 4)'

    # Trigger 4: Coverage degradation
    rolling_coverage = recent['coverage'].mean()
    if rolling_coverage < 65:
        return True, f'Coverage = {rolling_coverage:.0f}% (threshold: 65%)'

    return False, f'Performance OK — MAPE={rolling_mape:.1f}%, Bias={rolling_bias:+.0f}'


### 5.5 Fallback Strategy


In [ ]:
def forecast_with_fallback(df, holidays_df, horizon=7):
    """
    Layered forecasting with automatic fallback.
    
    Level 1: Prophet with tuned parameters and regressors
    Level 2: Prophet with default parameters (more robust)
    Level 3: 4-week same-day-of-week average (always works)
    """

    # Level 1: Full Prophet model
    try:
        logger.info('Attempting Level 1: Tuned Prophet...')
        model, forecast = build_and_forecast(df, holidays_df, horizon)
        sanity_check_forecast(forecast, df)
        logger.info('Level 1 SUCCESS')
        return forecast, 'prophet_tuned'

    except Exception as e:
        logger.warning(f'Level 1 FAILED: {e}')

    # Level 2: Simple Prophet (no custom regressors)
    try:
        logger.info('Attempting Level 2: Default Prophet...')
        simple_model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            holidays=holidays_df
        )
        simple_model.add_country_holidays(country_name='IE')
        simple_df = df[['ds', 'y']].copy()
        simple_model.fit(simple_df)

        future = simple_model.make_future_dataframe(periods=horizon)
        forecast = simple_model.predict(future)
        forecast = forecast[forecast['ds'] > df['ds'].max()][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        forecast['yhat'] = forecast['yhat'].clip(lower=0)

        logger.info('Level 2 SUCCESS')
        return forecast, 'prophet_simple'

    except Exception as e:
        logger.warning(f'Level 2 FAILED: {e}')

    # Level 3: Naïve 4-week average (always works)
    logger.info('Falling back to Level 3: 4-week same-day average...')
    df_indexed = df.set_index('ds')

    forecast_dates = pd.date_range(df['ds'].max() + pd.Timedelta(days=1), periods=horizon, freq='D')
    naive_forecasts = []

    for date in forecast_dates:
        same_dow_dates = [date - pd.Timedelta(weeks=w) for w in range(1, 5)]
        vals = df_indexed.loc[df_indexed.index.isin(same_dow_dates), 'y']
        mean_val = vals.mean() if len(vals) > 0 else df['y'].tail(28).mean()
        std_val = vals.std() if len(vals) > 1 else df['y'].tail(28).std()

        naive_forecasts.append({
            'ds': date,
            'yhat': mean_val,
            'yhat_lower': mean_val - 1.28 * std_val,  # ~80% CI
            'yhat_upper': mean_val + 1.28 * std_val
        })

    forecast = pd.DataFrame(naive_forecasts)
    logger.info('Level 3 SUCCESS (naive fallback)')
    return forecast, 'naive_4week'


---


## Prophet-Specific Tuning Guide

| Parameter | What It Controls | Default | Tune When... | Range |
|---|---|---|---|---|
| `changepoint_prior_scale` | Trend flexibility | 0.05 | Trend looks too rigid or too wiggly | 0.001 – 0.5 |
| `seasonality_prior_scale` | Seasonal smoothness | 10 | Seasonal pattern is too smooth or overfitting | 0.01 – 25 |
| `holidays_prior_scale` | Holiday effect size | 10 | Holiday spikes under/overestimated | 0.01 – 25 |
| `seasonality_mode` | Additive vs multiplicative | additive | Seasonal amplitude varies with level | additive / multiplicative |
| `changepoint_range` | Where changepoints can occur | 0.8 | Recent trend shifts are important | 0.7 – 0.95 |
| `n_changepoints` | Number of potential changepoints | 25 | Trend has many/few level shifts | 10 – 50 |
| `yearly_seasonality` | Fourier order for annual cycle | auto (10) | Annual pattern is too smooth/jagged | 3 – 25 |
| `weekly_seasonality` | Fourier order for weekly cycle | auto (3) | Day-of-week pattern not captured | 3 – 10 |
| `interval_width` | Prediction interval width | 0.80 | Calibration is poor | 0.70 – 0.95 |
| `mcmc_samples` | Bayesian sampling for uncertainty | 0 (MAP) | PI calibration matters for staffing | 0 or 300+ |

**Tuning priority order for call volume:** `seasonality_mode` → `changepoint_prior_scale` → `holidays_prior_scale` → `weekly_seasonality` Fourier order → everything else.

---


## Known Prophet Limitations and Workarounds

| Limitation | Impact on Call Forecasting | Workaround |
|---|---|---|
| No autocorrelation modelling | Cannot capture "bad week carries into next week" momentum | Post-process residuals with AR(1), or ensemble with XGBoost |
| Sensitive to outliers in trend estimation | Single extreme day can create a false changepoint | Set outliers to NaN before fitting |
| Holiday effects are constant over time | A bank holiday in 2020 (COVID) had a different effect than in 2024 | Use recent-only training data, or add interaction regressors |
| Cannot model intermittent demand | If weekends have zero volume, Prophet may produce negative forecasts for weekends | Clip forecasts at zero; or model weekdays and weekends separately |
| Annual seasonality needs > 2 years of data | Fourier terms cannot be estimated with < 2 full annual cycles | Use `yearly_seasonality=False` and model annual effects via monthly regressors if short history |
| No built-in ensemble or model averaging | Prophet gives one forecast; no automatic model combination | Build your own ensemble: 60% Prophet + 40% XGBoost, or use prediction interval overlap as a confidence measure |

---


## Quick Reference — Complete Pipeline Checklist

```
□ EXPLORE
  □ Plot full history — identify trend, seasonality, anomalies
  □ Day-of-week box plots — confirm weekly pattern
  □ Monthly/annual patterns — identify secondary seasonality
  □ MSTL decomposition (periods 7 and 365)
  □ ACF/PACF — confirm lag 7 dominance
  □ Stationarity tests (ADF + KPSS) — informational for Prophet

□ PREPROCESS
  □ Create Prophet df with ds/y columns
  □ Handle missing days (interpolate or fill)
  □ Flag and NaN true outliers (keep holiday effects)
  □ Build holiday DataFrame with lower/upper windows
  □ Build banking regressors (direct debit, payday, weekend)
  □ Train/test split (hold out 8+ weeks)

□ MODEL
  □ Compute seasonal naïve baseline (MAPE target to beat)
  □ Configure Prophet (growth, seasonalities, mode)
  □ Add holidays and regressors
  □ Fit and inspect components
  □ Check changepoints — do they make sense?
  □ Generate 7-day forecast with prediction intervals

□ DIAGNOSE
  □ Residual time plot — no pattern
  □ Residual ACF — no significant spikes
  □ Residuals by day of week — no systematic bias
  □ Ljung-Box test — p > 0.05
  □ Prophet cross-validation (initial/period/horizon)
  □ MAPE by horizon — stable or degrading?
  □ MAPE by day of week — Monday worst?
  □ Prediction interval coverage — near 80%?

□ DEPLOY & MONITOR
  □ Hyperparameter grid search via cross-validation
  □ Production script with data quality checks
  □ Sanity checks on forecast before publishing
  □ Erlang C integration for staffing output
  □ Weekly accuracy report (MAE, MAPE, bias, coverage)
  □ Drift detection and retraining triggers
  □ 3-level fallback (tuned → simple → naïve)
```


*Case study prepared as a supplement to the Time Series Fundamentals and EDA study notes. Focused on Prophet for the workforce planning function at a retail bank contact centre. February 2026.*
